dW0=W1-W0

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import random
from scipy import stats
import time

### SABR model Monte Carlo simulation

Monte Carlo methods (explained in the word) to estimate E[Payoff] and Var[E[Payoff]]. I will simulate L option prices, where each option price (L) will consist on the average of M simulated payoffs. 

The properties for these simulations will be:
- Initial price of the stock/$F_0$: 250
- Initial volatility/ $\sigma_0$: 0.3
- Volvol/ $\alpha$: 0.2
- B: 1
- p: 0.15
- K: 270
- T: 1 year, with 1000 steps 


In [4]:
L=50 #simulated options
M=50#prices per option

#time
T=1
n=1000

#parameters 
initial_price=250
initial_vol=0.3
volvol=0.2 
Beta=1
p=0.0
K=270

#risk-free rate
r=0

### Accrued Monte Carlo

Simulating an asset F following the SABR model

$\sigma_t = \sigma_0 \exp(-(\alpha^2/2)t + \alpha Z_t) $

$dW_t= pdZ_t + \sqrt(1-p^2)dB_t$

$dF_t = \sigma_t F_t^\beta  dW_t  $

In [5]:
#initial_price, initial volatility, volvol, Beta, maturity, correlation
def SABR_accrued(initial_price, initial_vol, volvol, Beta,T, p):
    dt=T/n
    time=[0]*n
    sigma=[0]*n
    sigma[0]=initial_vol
    Zt=[0]*n
    Bt=[0]*n
    F=[0]*n
    F[0]=initial_price
    
    for i in range(1,n):
        #update time
        time[i]=time[i-1]+dt
        
        #generate sigma[i]
        Zt[i]=Zt[i-1]+np.random.normal(0,np.sqrt(dt))
        dZ=Zt[i]-Zt[i-1]
        
        dsigma=(volvol*sigma[i-1]*dZ)
        sigma[i]=sigma[i-1]+dsigma
        #sigma[i]=sigma[0]*np.exp((-((volvol**2)/2)*time[i])+(volvol*Zt[i]))
    
        #generate the correlated brownian dW
        Bt[i]=Bt[i-1]+np.random.normal(0,np.sqrt(dt)) 
        dB=Bt[i]-Bt[i-1]
        dW=(p*dZ + np.sqrt(1-p**2)*dB)
        
        #generate the price F
        dF=sigma[i-1]*(F[i-1]**Beta)*dW
        F[i]=F[i-1]+dF
                       
    return F

In [6]:
#simulation of M payoffs, L times for each M payoff
last_price=np.zeros(M) #array of M last stock prices
payoff=np.zeros(M) #array of M payoffs

mean_pay=np.zeros(L) #an array of size L with the average payoff. Taking L time the average payoff of M simulations

start = time.time() #timer

for x in range(L):
    for i in range(M):
        #compute M stock paths
        last_price[i]=SABR_accrued(initial_price, initial_vol, volvol, Beta, T, p)[-1]
        
    #computing payoff of the M stock prices
    payoff=last_price-K
    payoff[payoff<0]=0
    
    #present value of the average payoff
    mean_pay[x]=np.exp(-r*T)*np.mean(payoff)

end = time.time()

print(end - start)

20.934510707855225


Mean and variance of the option payoff

In [7]:
print(np.mean(mean_pay))
print(np.std(mean_pay))

22.678443238834987
6.132768646993872


### Control variate Method

$\theta_c = Y + c (Z - E(Z))$

Where 

$Y= h(F_T)=max(F_T-K,0)$ 

$Z= F_T $

$E(Z)= F_0 exp(rT)=F_0$

$c=Cov(h(F_T),F_T)/Var(F_t)$


In [8]:
payoff=np.zeros(M)
last_price=np.zeros(M)
mean_pay=np.zeros(L)

start = time.time()
for x in range(L):
    for i in range(M):
        #compute M paths
        last_price[i]=SABR_accrued(initial_price, initial_vol, volvol, Beta,T, p)[-1]

    #computing payoff
    payoff=last_price-K
    payoff[payoff<0]=0
    
    #control variate
    Y=np.exp(-r*T)*payoff #present value of the payoffs
    Z=last_price #our control variate
    c=-np.cov(Y,Z)[0][1]/np.var(Z)
    control_variate=Y+c*(Z-(initial_price*np.exp(r*T)))
    
    mean_pay[x]=np.mean(control_variate)

end = time.time()

print(end-start)

20.63029909133911


In [9]:
print(np.mean(mean_pay))
print(np.std(mean_pay))

20.800787157643544
3.110218722212828


### Antithetic Variates Method

$\hat{\theta} = (Y + Y2)/2$

Where 

$Y= h(F_T)$ 

$Y2= h(F_Tneg)$ 


In [10]:
def SABR_antithetic(initial_price, initial_vol, volvol, Beta,T, p):
    T=T
    n=1000
    dt=T/n
    
    time=[0]*n
    Zt=[0]*n
    Bt=[0]*n
    
    sigma=[0]*n
    sigma[0]=initial_vol
    
    sigma_neg=[0]*n #antithetic
    sigma_neg[0]=initial_vol #antithetic
    
    F=[0]*n
    F[0]=initial_price   
    F_neg=[0]*n #antithetic
    F_neg[0]=initial_price #antithetic
    
    for i in range(1,n):
        time[i]=time[i-1]+dt # update time
        
        #generate volatility processes 
        Zt[i]=Zt[i-1]+np.random.normal(0,np.sqrt(dt))
        dZ=Zt[i]-Zt[i-1]
        
        dsigma=(volvol*sigma[i-1]*(dZ))
        sigma[i]=sigma[i-1]+dsigma  
        
        dsigma_neg=(volvol*sigma_neg[i-1]*(-dZ))
        sigma_neg[i]=sigma_neg[i-1]+dsigma_neg
        
        #correlated brownian motion dWt and its antithetic
        Bt[i]=Bt[i-1]+np.random.normal(0,np.sqrt(dt))
        dB=Bt[i]-Bt[i-1]
        dW=p*dZ + np.sqrt(1-p**2)*dB
        dW_neg=p*-dZ + np.sqrt(1-p**2)*-dB
        
        #price generation
        dF=(sigma[i-1]*(F[i-1]**Beta)*(dW))
        dF_neg=(sigma_neg[i-1]*(F_neg[i-1]**Beta)*(dW_neg))
        
        F[i]=F[i-1]+dF
        F_neg[i]=F_neg[i-1]+dF_neg
                
            
    return F[-1],F_neg[-1]

In [11]:
#simulation for M, L times
payoff=np.zeros(M)
last_price=np.zeros(M)
last_price_antithetic=np.zeros(M)
mean_pay=np.zeros(L)

start = time.time()
for x in range(L):
    for i in range(M):
        #compute M paths
        last_price[i],last_price_antithetic[i]=SABR_antithetic(initial_price, initial_vol, volvol, Beta,T, p)

    #computing payoff
    payoff=np.exp(-r*T)*np.maximum(last_price-K,0)
    
    #computing antithetic payoff
    antithetic_payoff=np.exp(-r*T)*np.maximum(last_price_antithetic-K,0)
    
    #estimator
    estimator=(payoff+antithetic_payoff)/2
    mean_pay[x]=np.mean(estimator)
    
end = time.time()

print(end-start)

25.23353910446167


In [12]:
print(np.mean(mean_pay))
print(np.std(mean_pay))

20.946893549904594
3.4746936625848384


### Conditional

In [21]:
#function for computing Black Scholes price
def bs_price(vol, initial_price, r, T, K):
    d1=np.log(initial_price/K) + (r+(vol*vol)/2)*T
    d1=d1/(vol*np.sqrt(T))

    d2=d1- vol*np.sqrt(T)

    vanilla_price=initial_price*stats.norm.cdf(d1) - K*np.exp(-r*T)*stats.norm.cdf(d2)
    return vanilla_price

#volatility process and Zt
def vol_Zt(initial_vol, volvol, T):
    n=1000
    dt=T/n
    
    time=[0]*n
    sigma=[0]*n
    sigma[0]=initial_vol
    Zt=[0]*n
    dZ=[0]*n

    #generate the volatility process
    for i in range(1,n):
        time[i]=time[i-1]+dt
        Zt[i]=Zt[i-1]+np.random.normal(0,np.sqrt(dt))
        dZ[i-1]=Zt[i]-Zt[i-1]
        
        dsigma=(volvol*sigma[i-1]*dZ[i-1])
        sigma[i]=sigma[i-1]+dsigma
        #sigma[i]=sigma[0]*np.exp((-((volvol**2)/2)*time[i])+(volvol*Zt[i]))
    return np.array(sigma),Zt,dZ

In [22]:
payoff=np.zeros(M)
mean_pay=np.zeros(L)

start = time.time()
for x in range(L):
    for i in range(M):
    #sigma and brownian motion
        sigma, Zt, dZ=vol_Zt(initial_vol, volvol, T)
        dt=T/n

        #Hull White formula for conditional expectation
        sigma_2=sigma**2
        v0=sigma_2.sum()*dt 
        v0=np.sqrt(v0/T)

        E=p*sum(sigma*dZ)-(((p**2)/2)*v0**2)*T
        S0=initial_price*np.exp(E)

        #vol, initial_price, r, T, K
        payoff[i]=bs_price(v0*np.sqrt(1-p**2),S0,0,T, K)
    mean_pay[x]=np.mean(payoff)

end = time.time()
print(end - start)

9.716392517089844


In [23]:
print(np.mean(mean_pay))
print(np.std(mean_pay))

22.11853810196485
0.45333546553379156


### Antithetic + Conditional Method

In [24]:
def SABR_CD_A(initial_vol, volvol, T):
    n=1000
    dt=T/n 
    time=[0]*n
    
    sigma1=[0]*n
    sigma1[0]=initial_vol
    Zt1=[0]*n
    dZ1=[0]*n
    
    sigma2=[0]*n
    sigma2[0]=initial_vol
    Zt2=[0]*n
    dZ2=[0]*n   

    #generate the volatility process and its antithetic
    for i in range(1,n):
        time[i]=time[i-1]+dt
        
        Zt1[i]=Zt1[i-1]+np.random.normal(0,np.sqrt(dt))
        dZ1[i-1]=Zt1[i]-Zt1[i-1]
        dZ2[i-1]=-dZ1[i-1]
        Zt2[i]=Zt2[i-1]+dZ2[i-1]
        
        #sigma1[i]=sigma1[0]*np.exp((-((volvol**2)/2)*time[i])+(volvol*Zt1[i]))
        #sigma2[i]=sigma2[0]*np.exp((-((volvol**2)/2)*time[i])+(volvol*Zt2[i]))
        dsigma1=(volvol*sigma1[i-1]*dZ1[i-1])
        sigma1[i]=sigma1[i-1]+dsigma1
        
        dsigma2=(volvol*sigma2[i-1]*dZ2[i-1])
        sigma2[i]=sigma2[i-1]+dsigma2
        
    return np.array(sigma1),Zt1,dZ1, np.array(sigma2),Zt2,dZ2

In [25]:
#simulation for M, L times
payoff=np.zeros(M)
mean_pay=np.zeros(L)

start = time.time()
for x in range(L):
    for i in range(M):
        #compute M paths
        sigma1,Zt1,dZ1,sigma2,Zt2,dZ2=SABR_CD_A(initial_vol, volvol, T)
        dt=T/n
        #####1: first payoff
        sigma1_2=sigma1**2
        v0=sigma1_2.sum()*dt
        v0=np.sqrt(v0/T)
        
        E=p*sum(sigma1*dZ1)-(((p**2)/2)*v0**2)*T
        S0=initial_price*np.exp(E)
        
        #vol, initial_price, r, T, K
        pay1=bs_price(v0*np.sqrt(1-p**2),S0,0,T, K)
    
        #####2: antithetic payoff
        sigma2_2=sigma2**2
        v0=sigma2_2.sum()*dt
        v0=np.sqrt(v0/T)
        
        E=p*sum(sigma2*dZ2)-(((p**2)/2)*v0**2)*T
        S0=initial_price*np.exp(E)
        
        #vol, initial_price, r, T, K
        pay2=bs_price(v0*np.sqrt(1-p**2),S0,0,T, K) 
        
        #antithetic payoff
        payoff[i]=(pay1+pay2)/2
        
    mean_pay[x]=np.mean(payoff)

end = time.time()

print(end - start)

11.375790357589722


In [26]:
print(np.mean(mean_pay))
print(np.std(mean_pay))

22.120255553707285
0.059701046942053165


### Control variate + Antithetics + Conditional

In [27]:
#simulation for M, L times
payoff=np.zeros(M)
last_vol=np.zeros(M)
mean_pay=np.zeros(L)

start = time.time()
for x in range(L):
    for i in range(M):
        #compute M paths
        sigma1,Zt1,dZ1,sigma2,Zt2,dZ2=SABR_CD_A(initial_vol, volvol, T)
        
        dt=T/n
        #####1: first payoff
        sigma1_2=sigma1**2
        v0=sigma1_2.sum()*dt
        v0=np.sqrt(v0/T)
        
        E=p*sum(sigma1*dZ1)-(((p**2)/2)*v0**2)*T
        S0=initial_price*np.exp(E)
        
        #vol, initial_price, r, T, K
        pay1=bs_price(v0*np.sqrt(1-p**2),S0,0,T, K)
    
        #####2: antithetic payoff
        sigma2_2=sigma2**2
        v0=sigma2_2.sum()*dt
        v0=np.sqrt(v0/T)
        
        E=p*sum(sigma2*dZ2)-(((p**2)/2)*v0**2)*T
        S0=initial_price*np.exp(E)
        
        #vol, initial_price, r, T, K
        pay2=bs_price(v0*np.sqrt(1-p**2),S0,0,T, K) 
        
        #antithetic payoff
        payoff[i]=(pay1+pay2)/2
        
        #mean of last value of volatilities as control variate
        last_vol[i]=(sigma1[-1]**2+sigma2[-1]**2)/2
    
    #antithetics
    Y=payoff
    Z=last_vol
    c=-np.cov(Y,Z)[0][1]/np.var(Z)
    
    
    expected_sigma=initial_vol
    var_sigma=initial_vol**2*(np.exp(volvol*volvol*T)-1)
    expected_sigma2=var_sigma+(expected_sigma**2)
    
    #computing new estimator (vector of M estimations)
    control_variate=Y+c*(Z-expected_sigma2)

    #now take mean of the M estimators to compute the Xth price
    mean_pay[x]=np.mean(control_variate)

end = time.time()

print(end - start)

11.486400842666626


In [28]:
print(np.mean(mean_pay))
print(np.std(mean_pay))

22.112443112839436
0.03315541226084339
